In [1]:
#!pip install wget

# importing the requests module
import requests
import wget

from os import rename
from os.path import exists


# importing the zipfile module
from zipfile import ZipFile
  
import pyodbc

import pandas as pd
import numpy as np


In [62]:
import warnings

warnings.filterwarnings('ignore')


In [2]:
dicFiles = {}
dicParams = {}
dicParams['dataFolder'] = "C:/ENES/"
dicParams['dataFolderZIP'] = dicParams['dataFolder'] + "ZIP/"
dicParams['dataFolderMDB'] = dicParams['dataFolder'] + "MDB/"
dicParams["FirstRun"] = False



dicFiles['ENES2008.zip'] =  'https://www.dge.mec.pt/sites/default/files/JNE/enes2008_0.zip'
dicFiles['ENES2009.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2009.zip'
dicFiles['ENES2010.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2010_enes.zip'

dicFiles['ENES2011.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2011_enes.zip'
dicFiles['ENES2012.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2012_enes_secun.zip'
dicFiles['ENES2013.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2013_enes_sec.zip'
dicFiles['ENES2014.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2014.zip'
dicFiles['ENES2015.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2015_0.zip'
dicFiles['ENES2016.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2016-media_2.zip'
dicFiles['ENES2017.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2017-final.zip'
dicFiles['ENES2018.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2018.zip'
dicFiles['ENES2019.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2019.zip'

dicFiles['ENES2020.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes_2020.zip'
dicFiles['ENES2021.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2021.zip'
dicFiles['ENES2022.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/enes2022_bd_imprensa.zip'


dicFiles['ENEB2008.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2008-itens_1.zip'
dicFiles['ENEB2009.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2009.zip'
dicFiles['ENEB2010.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2010_eneb.zip'

dicFiles['ENEB2011.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2011_eneb.zip'
dicFiles['ENEB2012.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2012_eneb_eb.zip'
dicFiles['ENEB2013.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/2013_eneb_2_3ciclos.zip'
dicFiles['ENEB2014.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2014.zip'
dicFiles['ENEB2015.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2015.zip'
dicFiles['ENEB2016.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2016.zip'
dicFiles['ENEB2017.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2017-final.zip'
dicFiles['ENEB2018.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2018_final_1.zip'
dicFiles['ENEB2019.zip'] =  'http://www.dge.mec.pt/sites/default/files/JNE/eneb2019_0.zip'



In [3]:
if dicParams["FirstRun"] == True:
    # loading the temp.zip and creating a zip object

    # download files in dicFiles from the Internet
    for key in dicFiles.keys():
        if not exists(dicParams['dataFolderZIP'] + key):
            print("Downloading " + key)
            wget.download(dicFiles[key], dicParams['dataFolderZIP'] + key)



In [4]:
if dicParams["FirstRun"] == True:
    # loading the temp.zip and creating a zip object


    for key in dicFiles.keys():
        zipfile = dicParams['dataFolderZIP'] + key
        
        with ZipFile(zipfile, 'r') as zObject:
            path=dicParams['dataFolderMDB'] + key[:-4] + ".mdb"
 
            filename = ZipFile.namelist(zObject)[0]

            # Extracting specific file in the zip
            # into a specific location.
            zObject.extract( member=filename, path=dicParams['dataFolderMDB'])


            print("Rename {} to {}".format(dicParams['dataFolderMDB'] +filename, dicParams['dataFolderMDB'] + key[:-4] + ".mdb"))
            rename(dicParams['dataFolderMDB'] + filename, dicParams['dataFolderMDB'] + key[:-4] + ".mdb")




In [6]:
# Replace with your Access database connection string
mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(2022) + '.mdb'
#connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile + '.accdb'
connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile

# Establish a connection to the database
connection = pyodbc.connect(connection_string)
cursor = connection.cursor()
cursorfields = connection.cursor()

# List tables
tables = cursor.tables(tableType='TABLE')
for table in tables:
    table_name = table.table_name
    print(f"Table: {table_name}")
    
    # List fields in the table
    # Execute a query to get the column names
    cursorfields.execute(f"SELECT * FROM {table_name} WHERE 1=0")  # This query retrieves no rows, but gets the table's structure
    column_names = [column[0] for column in cursorfields.description]

    # List the column names
    for column_name in column_names:
        #    print("        {}".format(column_name))
        pass

# List views
views = cursor.tables(tableType='VIEW')
for view in views:
    view_name = view.table_name
    print(f"View: {view_name}")
    
    # List fields in the view
    columns = cursor.columns(table_name=view_name)
    for column in columns:
        field_name = column.column_name
        print(f"  Field: {field_name}")

# Close the connection
connection.close()






Table: tblCodsConcelho
Table: tblCodsDistrito
Table: tblCodsPubPriv
Table: tblCodsSitFreq
Table: tblCursos
Table: tblCursosSubTipos
Table: tblCursosTipos
Table: tblEscolas
Table: tblExames
Table: tblHomologa_2022


# Distrito

# if 'dfDistrito' not in locals():
dfDistrito = pd.DataFrame( {'ano': [], 'Distrito': [], 'descr': []})


for ano in range( 2008, 2022+1):
	
	mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(ano) + '.mdb'
	connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile

	# Establish a connection to the database
	connection = pyodbc.connect(connection_string)

	# Create a cursor object to interact with the database
	cursor = connection.cursor()

	# Execute SQL query
	SQL = 'SELECT ' + str(ano) + ' as ano, * FROM tblCodsDistrito;'
	cursor.execute(SQL)

	ds = cursor.fetchall()



	# Create a Pandas dataframe from the data
	df = pd.DataFrame.from_records(ds, columns=[col[0] for col in cursor.description])


	dfDistrito = dfDistrito.append(df, ignore_index=True)


	#dsCodsDistrito[ano] = df

	# Close the cursor and connection
	cursor.close()
	connection.close()



#print(dsCodsDistrito)

dfDistrito.to_excel(dicParams['dataFolder'] + "dsCodsDistrito.xlsx")





# Concelho

dfConcelho = pd.DataFrame( {'ano': [], 'Distrito':[], 'Concelho': [], 'Descr': [], 'Nuts3': []})


for ano in range( 2008, 2022+1):
	
	mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(ano) + '.mdb'
	connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile

	# Establish a connection to the database
	connection = pyodbc.connect(connection_string)

	# Create a cursor object to interact with the database
	cursor = connection.cursor()

	# Execute SQL query
	SQL = 'SELECT ' + str(ano) + ' as ano, * FROM tblCodsConcelho;'
	cursor.execute(SQL)
	ds = cursor.fetchall()

	# Create a Pandas dataframe from the data
	df = pd.DataFrame.from_records(ds, columns=[col[0] for col in cursor.description])

	dfConcelho = dfConcelho.append(df, ignore_index=True)

	# Close the cursor and connection
	cursor.close()
	connection.close()



dfConcelho.to_excel(dicParams['dataFolder'] + "dsCodsConcelho.xlsx", index=False)



# Correções aos nomes dos Concelhos
dfCorrectConcelhoFromTo = {}
dfCorrectConcelhoFromTo['Feira'] = 'Santa Maria da Feira'
dfCorrectConcelhoFromTo['Terras do Bouro'] = 'Terras de Bouro'
dfCorrectConcelhoFromTo['Castanheira de Pera'] = 'Castanheira de Pêra'
dfCorrectConcelhoFromTo['Ponte de Sôr'] = 'Ponte de Sor'
dfCorrectConcelhoFromTo['Calheta'] = 'Calheta (R.A.A.)'
dfCorrectConcelhoFromTo['Vila da Praia da Vitória'] = 'Praia da Vitória'
dfCorrectConcelhoFromTo['Vila Praia da Vitória'] = 'Praia da Vitória'
dfCorrectConcelhoFromTo['Lages das Flores'] = 'Lajes das Flores'
dfCorrectConcelhoFromTo['Lages do Pico'] = 'Lajes do Pico'
dfCorrectConcelhoFromTo['Lagoa (Ponta Delgada)'] = 'Lagoa (R.A.A)'
dfCorrectConcelhoFromTo['Calheta (Funchal)'] = 'Calheta (R.A.M.)'






# PubPriv

dfPubPriv = pd.DataFrame( {'PubPriv': [], 'Descr': []})


for ano in range( 2008, 2022+1):
	
	mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(ano) + '.mdb'
	connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile

	# Establish a connection to the database
	connection = pyodbc.connect(connection_string)

	# Create a cursor object to interact with the database
	cursor = connection.cursor()

	# Execute SQL query
	SQL = 'SELECT ' + str(ano) + ' as ano, * FROM tblCodsPubPriv;'
	cursor.execute(SQL)
	ds = cursor.fetchall()

	# Create a Pandas dataframe from the data
	df = pd.DataFrame.from_records(ds, columns=[col[0] for col in cursor.description])

	dfPubPriv = dfPubPriv.append(df, ignore_index=True)

	# Close the cursor and connection
	cursor.close()
	connection.close()



dfPubPriv.to_excel(dicParams['dataFolder'] + "dsPubPriv.xlsx", index=False)



# SitFreq

dfAllYears = pd.DataFrame( {'SitFreq': [], 'Descr': []})


for ano in range( 2008, 2022+1):
	
	try:
		mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(ano) + '.mdb'
		connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile

		# Establish a connection to the database
		connection = pyodbc.connect(connection_string)

		# Create a cursor object to interact with the database
		cursor = connection.cursor()

		# Execute SQL query
		SQL = 'SELECT ' + str(ano) + ' as ano, * FROM tblCodsSitFreq;'
		cursor.execute(SQL)
		ds = cursor.fetchall()

		# Create a Pandas dataframe from the data
		df = pd.DataFrame.from_records(ds, columns=[col[0] for col in cursor.description])

		dfAllYears = dfAllYears.append(df, ignore_index=True)

		# Close the cursor and connection
		cursor.close()
		connection.close()
	except:
		print("Ano:",ano," - Erro")


dfAllYears.to_excel(dicParams['dataFolder'] + "dsSitFreq.xlsx", index=False)



# dfGeo will store information on Concelho, Distrito and Nut3

In [63]:
# Distrito+Concelho+Nuts

dfLocation = pd.DataFrame( {'Distrito':[], 'DescrDistrito':[], 'Concelho': [], 'DescrConcelho': [], 'Nuts3': [], 'Nuts3Descr': []})

# Get Nuts3 names from 2018 database

# Establish a connection to the database
mdbfile = dicParams['dataFolderMDB'] +  'ENES2018.mdb'
connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile
connection = pyodbc.connect(connection_string)

# Execute SQL query
SQL = 'SELECT Nuts3, Descr as DescrNuts3 FROM tblNuts3;'
dsNuts3 = pd.read_sql(SQL, connection)

# Get Distrito+Concelho+Nuts from the 2021 database

# Establish a connection to the database
mdbfile = dicParams['dataFolderMDB'] +  'ENES2021.mdb'
connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile
connection = pyodbc.connect(connection_string)

# Execute SQL query
SQL = 'SELECT conc.Distrito, distr.Descr as DescrDistrito, conc.Concelho, conc.Descr as DescrConcelho, conc.Nuts3 FROM tblCodsConcelho conc inner join tblCodsDistrito distr on conc.distrito = distr.distrito;'

dsGeo = pd.read_sql(SQL, connection)

# Close the cursor and connection
connection.close()

dsGeo = dsGeo.merge(dsNuts3, on="Nuts3", how="left")

del dsNuts3

dsGeo


,Distrito,DescrDistrito,Concelho,DescrConcelho,Nuts3,DescrNuts3
0,01,Aveiro,01,Águeda,16D,Região de Aveiro
1,01,Aveiro,02,Albergaria-a-Velha,16D,Região de Aveiro
2,01,Aveiro,03,Anadia,16D,Região de Aveiro
3,01,Aveiro,04,Arouca,11A,Área Metropolitana do Porto
4,01,Aveiro,05,Aveiro,16D,Região de Aveiro
...,...,...,...,...,...,...
304,22,R. A. Madeira,08,Ribeira Brava,300,Região Autónoma da Madeira
305,22,R. A. Madeira,09,Santa Cruz,300,Região Autónoma da Madeira
306,22,R. A. Madeira,10,Santana,300,Região Autónoma da Madeira
307,22,R. A. Madeira,11,São Vicente,300,Região Autónoma da Madeira


In [19]:
ds2

,Distrito,DescrDistrito,Concelho,DescrConcelho,Nuts3,DescrNuts3,Nuts3Descr
0,01,Aveiro,01,Águeda,16D,...,Região de Aveiro
1,01,Aveiro,02,Albergaria-a-Velha,16D,...,Região de Aveiro
2,01,Aveiro,03,Anadia,16D,...,Região de Aveiro
3,01,Aveiro,04,Arouca,11A,...,Área Metropolitana do Porto
4,01,Aveiro,05,Aveiro,16D,...,Região de Aveiro
...,...,...,...,...,...,...,...
304,22,R. A. Madeira,08,Ribeira Brava,300,...,Região Autónoma da Madeira
305,22,R. A. Madeira,09,Santa Cruz,300,...,Região Autónoma da Madeira
306,22,R. A. Madeira,10,Santana,300,...,Região Autónoma da Madeira
307,22,R. A. Madeira,11,São Vicente,300,...,Região Autónoma da Madeira


In [66]:
#Escola

dfAllYears = pd.DataFrame( {'SitFreq': [], 'Descr': []})


for ano in range( 2008, 2022+1):
	
	try:
		mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(ano) + '.mdb'
		connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile

		# Establish a connection to the database
		connection = pyodbc.connect(connection_string)

		# Create a cursor object to interact with the database
		cursor = connection.cursor()

		# Execute SQL query

		SQL = "CREATE VIEW vwEscolas as SELECT tblEscolas.Distrito, tblEscolas.Concelho, tblEscolas.Escola, tblEscolas.Descr, tblEscolas.PubPriv FROM tblEscolas;"
		SQL = "SELECT " + str(ano) + " as ano, Distrito, Concelho, Escola, Descr, PubPriv, CodDGEEC FROM tblEscolas;"


		cursor.execute(SQL)
		ds = cursor.fetchall()

		# Create a Pandas dataframe from the data
		df = pd.DataFrame.from_records(ds, columns=[col[0] for col in cursor.description])

		dfAllYears = dfAllYears.append(df, ignore_index=True)

		# Close the cursor and connection
		cursor.close()
		connection.close()
	except Exception as ex:
		print("Ano:",ano," - Erro: ", ex)


dfAllYears.to_excel(dicParams['dataFolder'] + "Escolas.xlsx", index=False)




Ano: 2008  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')
Ano: 2009  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')
Ano: 2010  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')
Ano: 2011  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')
Ano: 2012  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')
Ano: 2013  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')
Ano: 2014  - Erro:  ('07002', '[07002] [Microsoft][ODBC Microsoft Access Driver] Too few parameters. Expected 1. (-3010) (SQLExecDirectW)')


In [67]:
#Escola


# LOAD 2022

# Establish a connection to the database
mdbfile = dicParams['dataFolderMDB'] +  'ENES2022.mdb'
connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile
connection = pyodbc.connect(connection_string)

# Create a cursor object to interact with the database
#cursor = connection.cursor()

# Execute SQL query
SQL = "SELECT 2022 as AnoDadosEscola, Distrito, Concelho, Escola, Descr, PubPriv, CodDGEEC FROM tblEscolas;"
dfSchools = pd.read_sql(SQL, connection)

# Close the connection
connection.close()


# Now loop throught the previous years, in descending order, to get schools that no longer exist

for ano in range( 2022-1, 2008-1, -1):
	
	try:


		# Establish a connection to the database
		mdbfile = dicParams['dataFolderMDB'] +  'ENES' + str(ano) + '.mdb'
		connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile
		connection = pyodbc.connect(connection_string)

		# Execute SQL query
		if ano >= 2015:
			SQL = "SELECT " + str(ano) + " as AnoDadosEscola, Distrito, Concelho, Escola, Descr, PubPriv, CodDGEEC FROM tblEscolas;"
		else:
			SQL = "SELECT " + str(ano) + " as AnoDadosEscola, Distrito, Concelho, Escola, Descr, PubPriv, distrito * 100000 + concelho * 1000 + 999 as CodDGEEC FROM tblEscolas;"
		

		dfUpdates = pd.read_sql(SQL, connection)

				
		# Identify rows in dfUpdates that don't exist in dfSchools
		mask = ~dfUpdates[['Distrito', 'Concelho', 'Escola']].apply(tuple, axis=1).isin(dfSchools[['Distrito', 'Concelho', 'Escola']].apply(tuple, axis=1))

		# Filter rows in dfUpdates based on the mask
		dfUpdates = dfUpdates[mask]


		# Concatenate dfSchools and the filtered new_rows
		#result = pd.concat([dfSchools, new_rows])
		dfSchools = pd.concat([dfSchools, dfUpdates])

		print(ano)
		print("Adding dfUpdates shape: ", dfUpdates.shape)
		print("Resulting dfSchools shape: ", dfSchools.shape)

		# Reset the index in the result DataFrame
		#result.reset_index(drop=True, inplace=True)
		dfSchools.reset_index(drop=True, inplace=True)

		#dfEscolas = dfEscolas.append(df, ignore_index=True)



		# Close the connection
		connection.close()
	except Exception as ex:
		print("Ano:",ano," - Erro: ", ex.message)





2021
Adding dfUpdates shape:  (3, 7)
Resulting dfSchools shape:  (650, 7)
2020
Adding dfUpdates shape:  (3, 7)
Resulting dfSchools shape:  (653, 7)
2019
Adding dfUpdates shape:  (6, 7)
Resulting dfSchools shape:  (659, 7)
2018
Adding dfUpdates shape:  (8, 7)
Resulting dfSchools shape:  (667, 7)
2017
Adding dfUpdates shape:  (8, 7)
Resulting dfSchools shape:  (675, 7)
2016
Adding dfUpdates shape:  (3, 7)
Resulting dfSchools shape:  (678, 7)
2015
Adding dfUpdates shape:  (6, 7)
Resulting dfSchools shape:  (684, 7)
2014
Adding dfUpdates shape:  (4, 7)
Resulting dfSchools shape:  (688, 7)
2013
Adding dfUpdates shape:  (2, 7)
Resulting dfSchools shape:  (690, 7)
2012
Adding dfUpdates shape:  (4, 7)
Resulting dfSchools shape:  (694, 7)
2011
Adding dfUpdates shape:  (3, 7)
Resulting dfSchools shape:  (697, 7)
2010
Adding dfUpdates shape:  (6, 7)
Resulting dfSchools shape:  (703, 7)
2009
Adding dfUpdates shape:  (6, 7)
Resulting dfSchools shape:  (709, 7)
2008
Adding dfUpdates shape:  (15, 7)


In [71]:
#SitFreq


# LOAD SitFreq from 2018

# Establish a connection to the database
mdbfile = dicParams['dataFolderMDB'] +  'ENES2018.mdb'
connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile
connection = pyodbc.connect(connection_string)

# Execute SQL query
SQL = "SELECT SitFreq, Descr, Defin FROM tblCodsSitFreq;"
dfSitFreq = pd.read_sql(SQL, connection)

# Close the connection
connection.close()

print(dfSitFreq)



  SitFreq                        Descr  \
0      AE             Admitido a Exame   
1      AM             Anulou Matrícula   
2      AP                     Aprovado   
3      EF          Excluído por Faltas   
4      RF          Reprovou Frequência   
5      AF  Autoproposto sem Frequência   

                                               Defin  
0  Aluno interno que obteve pelo menos 10 valores...  
1  Aluno que frequentou no presente ano letivo a ...  
2  Aluno que aprovou por frequência em disciplina...  
3  Aluno que frequentou a disciplina na escola es...  
4  Aluno que frequentou a disciplina na escola at...  
5  Aluno autoproposto que não frequentou a discip...  


In [72]:
# Curso, tipo, subtipo


# LOAD SitFreq from 2018

# Establish a connection to the database
mdbfile = dicParams['dataFolderMDB'] +  'ENES2021.mdb'
connection_string = 'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + mdbfile
connection = pyodbc.connect(connection_string)

# Execute SQL query
SQL = "Select curso.Curso, curso.TpCurso, curso.SubTipo, curso.Descr as DescrCurso,"\
"tpcurso.TpCurso, tpcurso.Descr as DescrTipoCurso, tpcurso.Ano_Ini, tpcurso.Ano_Term, tpcurso.Ordena,"\
"subtipos.Descr as DescrSubtipoCurso "\
"From ( tblCursos curso inner join tblCursosTipos tpcurso on curso.TpCurso = tpcurso.TpCurso ) inner join tblCursosSubTipos subtipos on curso.SubTipo = subtipos.SubTipo;"

dfCursos = pd.read_sql(SQL, connection)

# Close the connection
connection.close()

print(dfCursos)





    Curso TpCurso SubTipo                                         DescrCurso  \
0     002       E     E01                                          2.º Curso   
1     003       E     E01                                          3.º Curso   
2     004       E     E01                                          4.º Curso   
3     005       E     E01                                          5.º Curso   
4     001       E     E01                                          1.º Curso   
..    ...     ...     ...                                                ...   
483   220       U     U01  Ens. Sec. Recorrente (Todos os Cursos Gerais e...   
484   225       U     U01  Ens. Sec. Recorrente (Todos os Cursos Gerais e...   
485   230       U     U01  Ens. Sec. Recorrente Particular e Coop. (Todos...   
486   604       V     V01  Cursos da Via Profissionalizante do 12º Ano (T...   
487   606       W     W01  Cursos da Via Profissionalizante dos 10º/12º A...   

    TpCurso                           D